In [1]:
from scipy.stats import multivariate_normal as mvn
from scipy.stats import norm
from scipy.stats import chi2
import numpy as np
import pandas as pd
from scipy import optimize
import math

In [10]:
# one asset VaR, simpliest 
alpha = 0.05
z_alpha = norm.ppf(alpha)

# !!! ANNUAL parameters
mean = 0.01 * 252
std = 0.04 * math.sqrt(252)

position = 100


# parameter end

VaR = - position * (mean + z_alpha * std)
print("annual VaR at", alpha, ":", round(VaR, 4))
VaR = - position * (mean/12 + z_alpha * std/math.sqrt(12))
print("monthly VaR at", alpha, ":", round(VaR, 4))
VaR = - position * (mean/252 + z_alpha * std/math.sqrt(252))
print("daily VaR at", alpha, ":", round(VaR, 4))

annual VaR at 0.05 : -147.555
monthly VaR at 0.05 : 9.1507
daily VaR at 0.05 : 5.5794


In [11]:
# 2 assets portfolio VaR
mean1 = 1/100
mean2 = 0.5/100

std1 = 4/100
std2 = 7/100
corr = 0.25

position1 = 100
position2 = 250 


z_alpha = norm.ppf(0.05)

# parameter end
p = np.array([position1, position2])
m = np.array([mean1, mean2])
v = np.array([[std1 ** 2, corr * std1 * std2],
              [corr * std1 * std2, std2 ** 2]])


position = p.sum()
mean = np.dot(p, m)
std = math.sqrt(np.dot(np.dot(p, v), p))
VaR = - (mean + z_alpha * std)
print("annual VaR of portfolio:", round(VaR, 4))

annual VaR of portfolio: 28.8395


In [13]:
# marginal VaR and beta approach
mean_A = 0.2
mean_B = 0.1

w_A = 0.8
w_B = 0.2

beta_A = 0.9
beta_B = 1.4

# beta1 = (w1*std1**2 + w2*rho*std1*std2)/(std_po**2)
# beta2 = (w2*std2**2 + w1*rho*std1*std2)/(std_po**2)

std_po = math.sqrt(0.07)

new_po_A = 200

z_alpha = norm.ppf(0.05)



mean_po = w_A * mean_A + w_B * mean_B

# marginal VaR
mVaR_A = - (mean_A + z_alpha * beta_A * std_po)
VaR_po = - (mean_po + z_alpha * std_po)
print("marginal VaR:", round(mVaR_A, 4))
print("portfolio VaR:", round(VaR_po, 4))
print("contribution of A to the total VaR:", round(mVaR_A*w_A/VaR_po, 4))
print()

# new VaR
comp = new_po_A * mVaR_A
print("new VaR:", round(comp, 4))

marginal VaR: 0.1917
portfolio VaR: 0.2552
contribution of A to the total VaR: 0.6009

new VaR: 38.3337


In [5]:
# significance test (LRl, LRu)
# N: number of total sample days
# T: number of total days when loss was bigger than VaR (violation)
# 01-seq: no/yes violation today(1st digit), and no/yes violation next day(2nd digit).

N = 10339
T = 364

T00 = 9656
T01 = 318
T10 = 318
T11 = 46

alpha = 0.05


p0 = (T00 + T01) / (T00 + T01 + T10 + T11)
p1 = (T10 + T11) / (T00 + T01 + T10 + T11)
p00 = T00 / (T00 + T01)
p01 = T01 / (T00 + T01)
p10 = T10 / (T10 + T11)
p11 = T11 / (T10 + T11)

LRu = 2 * math.log(((T/N)/alpha)**T * ((1-T/N)/(1-alpha))**(N-T))
LRl = 2 * math.log((p00/p0)**T00 * (p01/p1)**T01 * (p10/p0)**T10 * (p11/p1)**T11)

print("unconditional coverage (LR_u):", round(LRu, 4), "\t", "reject" if LRu>chi2.ppf(q=1-alpha, df=1) else "do not reject")
print("independence (LR_l):", round(LRl, 4), "\t", "reject" if LRl>chi2.ppf(q=1-alpha, df=1) else "do not reject")
print("total test (LR_TOT):", round(LRu+LRl, 4), "\t", "reject" if LRu+LRl>chi2.ppf(q=1-alpha, df=2) else "do not reject")

unconditional coverage (LR_u): 52.8927 	 reject
independence (LR_l): 57.7938 	 reject
total test (LR_TOT): 110.6865 	 reject


In [6]:
# beta hedge, with stock
position_exposure = 2040000
beta_exposure = 1.5

T = 4 / 12
p_index = 4300
num_per_cont = 250

rf = 6 / 100
q = 4 / 100

# simple stock hedge
h = beta_exposure
position_hedge = -h * position_exposure
print("hedge position with stock:", round(position_hedge, 4))
print()


# simple future hedge
value_future = p_index * math.exp(T * (rf - q))
num_cont = - position_exposure * beta_exposure / (p_index * num_per_cont)

print("Value of Future:", round(value_future, 4))
print("optical number of contracts:", round(num_cont, 4))

hedge position with stock: -3060000.0

Value of Future: 4328.7624
optical number of contracts: -2.8465


In [ ]:
# basis risk
# basis = S(A, T) - F(B, T)
# basis = asset mismatch + maturity mismatch
#       = (S(A, T) - S(B, T)) + (S(B, T) - F(B, T))

In [ ]:
# contango: E(St) < F0, the forward price is above expected spot price. E(rm) - rf < 0. profit on short forward is positive.
# backwardation: E(St) > F0, the forward price is below expected spot price, E(rm) - rf > 0. profit on long forward is positive.

In [7]:
# 2 assets hedge
# A: undercovered assets
# B: hedging instruments
# rho: correlation

position_A = -1000000   # will buy, so it is a short exposure

std_A = 0.032
std_B = 0.04
rho_AB = 0.8

num_per_cont_B = 42000


h = rho_AB * std_A / std_B
position_B = - position_A * h 

print("optimal hedge ratio:", round(h, 4))
print("position of B:", round(position_B, 4))
print("optical number of B contracts:", round(position_B / num_per_cont_B, 4))

optimal hedge ratio: 0.64
position of B: 640000.0
optical number of B contracts: 15.2381


In [8]:
# hedge ratio with different maturities
T1 = 1
T2 = 6

rf = 0.1

u = 0
y = 0

h = math.exp(-(rf + u - y) * (T2 - T1))
print("optimal hedge ratio:", round(h, 4))

# expected profit of futures: F0 * (1 - exp(beta(RM - Rf) * T))

optimal hedge ratio: 0.6065


In [9]:
# hedge Capital at Risk:
S = 100
rf = 0.06
rm = 0.12
beta = 1.5
std_S = 0.15

T_fwd = 3 / 12
T_expo = 2

z_alpha = norm.ppf(0.05)


mu = S * math.exp(rf * T_fwd) * (1 - math.exp(beta * (rm - rf) * T_fwd))
sig = S * std_S * math.sqrt(T_fwd)
CaR = - (mu + z_alpha * sig)
simCaR = - (z_alpha * sig)

print("CaR with mu:", round(CaR, 4))
print("CaR without mu:", round(simCaR, 4))

CaR with mu: 14.6463
CaR without mu: 12.3364
